In [108]:
import numpy as np

In [109]:
# Функция Розенброка
def f(x, y):
    
    return (1 - x) ** 2 + 100 * (y - x ** 2) ** 2

In [110]:
# Градиент минимизируемой функции
def df(x, y):
    
    return np.array([[2 * (x - 1) + 400 * x * (x ** 2 - y)], [200 * (y - x ** 2)]])

In [111]:
# матрица Гессе минимизируемой функции
def Hf(x, y):
    
    H11 = 2 + 400 * (x ** 2 - y) * (2 * x + 1)
    H12 = -400 * x
    H21 = -400 * x
    H22 = 200
    
    return np.array([[H11, H12], [H21, H22]])

In [112]:
# Ограничение - круговая область
def g(x, y):
    
    return np.array([[x ** 2 + y ** 2 - 2]])

In [113]:
# Матрица Якоби ограничивающей функции
def Jg(x, y):
    
    J11 = 2 * x
    J12 = 2 * y
    
    return np.array([[J11, J12]])

In [114]:
# Матрица Гессе ограничивающей функции
def Hg(x, y):
    
    H11 = 2
    H12 = 0
    H21 = 0
    H22 = 2
    
    return np.expand_dims(np.array([[H11, H12], [H21, H22]]), axis=1)

In [115]:
# Левый верхний блок матрицы Якоби для градиента функции Лагранжа
def B(x, y, Lambda, Hf, Hg):
    
    return Hf(x, y) + np.squeeze(np.dot(Lambda.T, Hg(x, y)), axis=0)
    

In [116]:
# Матрица Якоби для градиента функции Лагранжа
def JdL(x, y, Lambda):
    
    B_ = B(x, y, Lambda, Hf, Hg)
    Jg_ = Jg(x, y)
    m = Jg_.shape[0]
    
    JdL_up = np.hstack([B_, Jg_.T])
    JdL_bottom = np.hstack([Jg_, np.zeros((m, m))])
    
    return np.vstack([JdL_up, JdL_bottom])

In [122]:
# Градиент функции Лагранжа
def dL(x, y, Lambda):
    
    L1 = df(x, y) + Jg(x, y).T @ Lambda
    L2 = g(x, y)
    
    return np.vstack([L1, L2])

In [123]:
dL(0, 0, np.ones((1, 1)))

array([[-2.],
       [ 0.],
       [-2.]])

In [128]:
JdL(1, 1, np.ones((1, 1)))

array([[   4., -400.,    2.],
       [-400.,  202.,    2.],
       [   2.,    2.,    0.]])

In [81]:
np.dot(np.ones((2, 1)).T, np.ones((2, 2, 2))).shape

(1, 2, 2)

In [130]:
np.linalg.solve(JdL(1, 0, np.ones((1, 1))), -dL(1, 0, np.ones((1, 1))))

array([[   0.5       ],
       [   1.98019802],
       [-105.96039604]])

In [125]:
np.linalg.det(JdL(0, 0, np.ones((1, 1))))

np.float64(0.0)

In [132]:
float(JdL(0, 0, np.ones((1, 1)))[0][0])

4.0

In [137]:
def SQP_step(x, y, Lambda, JdL, dL):
    
    JdL_ = JdL(x, y, Lambda)
    dL_ = dL(x, y, Lambda)
    
    shift = np.linalg.solve(JdL_, -dL_)
    
    shift_x = shift[0].item()
    shift_y = shift[1].item()
    
    shift_Lambda = shift[2:, :]
    
    return shift_x, shift_y, shift_Lambda
    

In [138]:
SQP_step(1, 0, np.ones((1, 1)), JdL, dL)

(0.5000000000000002, 1.9801980198019804, array([[-105.96039604]]))

In [241]:
def SQP_method(x_0, y_0, Lambda_0, JdL, dL, goal=1e-5):
    
    x = x_0
    y = y_0
    Lambda = Lambda_0
    
    grad_L = np.sqrt(np.sum(dL(x, y, Lambda) ** 2))
    
    print(f"Шаг 0:\t x = {x},\t y = {y},\t f = {f(x, y)},\t |grad L| = {np.sqrt(np.sum(dL(x, y, Lambda) ** 2))}")
    
    i = 0
    
    while abs(grad_L) > goal:
        
        i += 1
        
        shift_x, shift_y, shift_Lambda = SQP_step(x, y, Lambda, JdL, dL)
        
        x += shift_x
        y += shift_y
        Lambda += shift_Lambda
        
        grad_L = np.sqrt(np.sum(dL(x, y, Lambda) ** 2))
        
        print(f"Шаг {i + 1}:\t x = {x:.5f},\t y = {y:.5f},\t f = {f(x, y):.4f},\t |grad L| = {grad_L:.5e}")
    
    print("\n_________________________________________________________________________________\n")
    print(f"\n// Найден локальный минимум за {i + 1} шагов //\n\n\
В точке:\t\t({x:.3f}, {y:.3f})\n\
Значение функции:\t\t {f(x, y):.3f}\n")
        
    return x, y

In [242]:
x, y = SQP_method(x_0=0.5, y_0=0.5, Lambda_0=0.5 * np.ones((1, 1)), JdL=JdL, dL=dL, goal=1e-5)

Шаг 0:	 x = 0.5,	 y = 0.5,	 f = 6.5,	 |grad L| = 71.43353554178877
Шаг 2:	 x = 2.23886,	 y = 0.26114,	 f = 2259.0786,	 |grad L| = 5.85665e+03
Шаг 3:	 x = 1.55648,	 y = 0.21291,	 f = 488.6014,	 |grad L| = 3.65710e+03
Шаг 4:	 x = 1.37554,	 y = 0.43672,	 f = 211.9554,	 |grad L| = 6.46522e+02
Шаг 5:	 x = 1.04953,	 y = 1.36872,	 f = 7.1430,	 |grad L| = 4.40146e+02
Шаг 6:	 x = 0.90506,	 y = 1.12336,	 f = 9.2641,	 |grad L| = 1.26211e+02
Шаг 7:	 x = 1.28293,	 y = 0.78283,	 f = 74.5709,	 |grad L| = 6.11967e+02
Шаг 8:	 x = 1.05357,	 y = 0.99346,	 f = 1.3612,	 |grad L| = 5.11298e+01
Шаг 9:	 x = 0.96888,	 y = 1.03447,	 f = 0.9174,	 |grad L| = 5.65316e+01
Шаг 10:	 x = 1.03885,	 y = 0.96465,	 f = 1.3139,	 |grad L| = 6.84097e+01
Шаг 11:	 x = 0.98122,	 y = 1.02165,	 f = 0.3467,	 |grad L| = 3.70270e+01
Шаг 12:	 x = 1.01987,	 y = 0.98132,	 f = 0.3464,	 |grad L| = 3.49396e+01
Шаг 13:	 x = 0.98774,	 y = 1.01312,	 f = 0.1406,	 |grad L| = 2.29422e+01
Шаг 14:	 x = 1.01164,	 y = 0.98881,	 f = 0.1199,	 |grad L